In [12]:
import os, sys

sys.path.insert(0, os.path.abspath("../../.."))

In [13]:
from models.xgboost.model_3.model import RegressionSklienModel
import pandas as pd
from pandas.plotting import scatter_matrix
from lib.data_splitter import split_features_and_target_into_train_and_test_set
from lib.data_util import load_candlesticks
import xgboost as xgb
from sklearn.metrics import mean_squared_error, make_scorer
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV


In [14]:
candlesticks = load_candlesticks("BTCUSDT", "1h", custom_tmp_path="../../../tmp")

candlesticks

,open time,open,high,low,close,volume,close time,quote asset volume,number of trades,taker buy base asset volume,taker buy quote asset volume
0,1502942400000,4261.48,4313.62,4261.32,4308.83,47.181009,1502945999999,2.023661e+05,171,35.160503,1.509525e+05
1,1502946000000,4308.83,4328.69,4291.37,4315.32,23.234916,1502949599999,1.003048e+05,102,21.448071,9.260828e+04
2,1502949600000,4330.29,4345.45,4309.37,4324.35,7.229691,1502953199999,3.128231e+04,36,4.802861,2.079532e+04
3,1502953200000,4316.62,4349.99,4287.41,4349.99,4.443249,1502956799999,1.924106e+04,25,2.602292,1.129135e+04
4,1502956800000,4333.32,4377.85,4333.32,4360.69,0.972807,1502960399999,4.239504e+03,28,0.814655,3.552747e+03
...,...,...,...,...,...,...,...,...,...,...,...
23252,1587020400000,6688.63,6818.46,6686.39,6780.00,3399.164609,1587023999999,2.300670e+07,22202,1901.518707,1.285952e+07
23253,1587024000000,6893.24,6923.52,6871.13,6888.30,1641.824781,1587027599999,1.132671e+07,12148,873.070968,6.024127e+06
23254,1587027600000,7045.30,7097.00,7017.00,7027.09,4302.835917,1587031199999,3.038567e+07,31923,2361.179694,1.667550e+07
23255,1587031200000,7027.70,7061.80,7026.77,7040.00,1512.420788,1587034799999,1.066239e+07,12131,938.969209,6.619599e+06


In [15]:
features = RegressionSklienModel.generate_features(candlesticks)
target = RegressionSklienModel.generate_target(features)

target

0        5.0
1        5.0
2        4.0
3        3.0
4        3.0
        ... 
23252    4.0
23253    3.0
23254    2.0
23255    1.0
23256    0.0
Name: target, Length: 23257, dtype: float64

In [16]:
target.describe()

count    23257.000000
mean         0.149374
std          2.627953
min         -5.000000
25%         -2.000000
50%          0.000000
75%          2.000000
max          5.000000
Name: target, dtype: float64

In [17]:
(
    training_set_features,
    training_set_target,
    test_set_features,
    test_set_target,
) = split_features_and_target_into_train_and_test_set(features, target, 20)

In [18]:
# scatte = scatter_matrix(test_set_features.iloc[:, -5:], c=test_set_target.iloc[:], s=40, hist_kwds={"bins": 15}, figsize=(20,20))

In [33]:
raw_model = xgb.XGBRegressor(objective="reg:squarederror")

param_dist = {'n_estimators': stats.randint(150, 1000),
              'learning_rate': stats.uniform(0.01, 0.6),
              'subsample': stats.uniform(0.3, 0.9),
              'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 25],
              'colsample_bytree': stats.uniform(0.5, 0.9),
              'min_child_weight': [1, 2, 3, 4, 5, 10]
             }
rmse = make_scorer(mean_squared_error, greater_is_better=False)

r = RandomizedSearchCV(raw_model, param_distributions=param_dist,
                       scoring=rmse,
                       n_iter=3,
                       n_jobs=2)
r.fit(training_set_features, training_set_target)

r.score(test_set_features, test_set_target)


XGBoostError: value 1.38482 for Parameter colsample_bytree exceed bound [0,1]
colsample_bytree: Subsample ratio of columns, resample on each tree construction.

In [28]:
r.best_params_


{'colsample_bytree': 0.716549113595231,
 'learning_rate': 0.05488437390157618,
 'max_depth': 3,
 'min_child_weight': 2,
 'n_estimators': 181,
 'subsample': 0.33243402437274977}

In [31]:
xg_boost = RegressionSklienModel(model=xgb.XGBRegressor(
    objective="reg:squarederror",
    colsample_bytree= 0.716549113595231,
    learning_rate= 0.05488437390157618,
    max_depth= 3,
    min_child_weight= 2,
    n_estimators= 181,
    subsample= 0.33243402437274977
    ))
xg_boost.train(training_set_features, training_set_target)
xg_boost.evaluate(test_set_features, test_set_target)

RMSE: 2.692856
kfold Accuracy: -10.37% (14.07%)


In [21]:
predictions = xg_boost.predict_dataframe(test_set_features)
test_set_target
df = pd.DataFrame({"target": test_set_target, "prediction": predictions})
df.to_csv("./tmp/target_predictions.csv")

NameError: name 'xg_boost' is not defined